# Persistence

Many AI applications need memory to share context across multiple interactions. In LangGraph, memory is provided for any [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.StateGraph) through [Checkpointers](https://langchain-ai.github.io/langgraph/reference/checkpoints/).

When creating any LangGraph workflow, you can set them up to persist their state by doing using the following:

1. A [Checkpointer](https://langchain-ai.github.io/langgraphjs/reference/classes/index.BaseCheckpointSaver.html), such as the [MemorySaver](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MemorySaver.html)
2. Call `compile(checkpointer=myCheckpointer)` when compiling the graph.

<!-- Example:
```javascript
from langgraph.graph import StateGraph
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

builder = StateGraph(....)
# ... define the graph
memory = AsyncSqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)
...
``` -->

This works for [StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html and all its subclasses, such as [MessageGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#messagegraph).

Below is an example.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        In this how-to, we will create our agent from scratch to be transparent (but verbose). You can accomplish similar functionality using the <code>create_react_agent(model, tools=tool, checkpointer=checkpointer)</code> (<a href="https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html">API doc</a>) constructor. This may be more appropriate if you are used to LangChain’s <a href="https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentexecutor">AgentExecutor</a> class.
    </p>
</div>


## Setup

This guide will use Anthropic's Claude model. We will optionally set our API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [54]:
// Deno.env.set("ANTHROPIC_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
Deno.env.set("LANGCHAIN_API_KEY", "ls__...");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Persistence: LangGraphJS");

## Define the state

The state is the interface for all of the nodes in our graph.

In [21]:
import { BaseMessage } from "@langchain/core/messages";

interface IState {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => BaseMessage[];
    default: () => BaseMessage[];
  };
  next: string;
}

// This defines the agent state
const State: IState = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
};

## Set up the tools

We will first define the tools we want to use. For this simple example, we will use create a placeholder search engine.
However, it is really easy to create your own tools - see documentation [here](https://js.langchain.com/v0.2/docs/how_to/custom_tools) on how to do that.

In [62]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({ query }: { query: string }) => {
    // This is a placeholder for the actual implementation
    return ["Cold, with a low of -78 ℃"];
  },
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool]

[ "Cold, with a low of -78 ℃" ]

We can now wrap these tools in a simple [ToolNode](https://langchain-ai.github.io/langgraphjs/reference/classes/prebuilt.ToolNode.html). This object will actually run the tools (functions) whenever they are invoked by our LLM.

In [63]:
// import { ToolNode } from "@langchain/langgraph/prebuilt";
import { ToolNode } from "../../langgraph/dist/prebuilt/index.js";
const toolNode = new ToolNode(tools);

## Set up the model

Now we will load the [chat model]().
https://js.langchain.com/v0.2/docs/concepts/#chat-models
1. It should work with messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.
2. It should work with [tool calling](https://js.langchain.com/v0.2/docs/how_to/tool_calling/#passing-tools-to-llms), meaning it can return function arguments in its response.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        These model requirements are not general requirements for using LangGraph - they are just requirements for this one example.
    </p>
</div>    

In [64]:
import { ChatAnthropic } from "@langchain/anthropic";

const model = new ChatAnthropic( {model: "claude-3-haiku-20240307"});

After we've done this, we should make sure the model knows that it has these tools available to call. We can do this by calling [bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools).

In [65]:
const boundModel =  model.bindTools(tools);
boundModel.kwargs

{
  tools: [
    {
      name: "search",
      description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
      input_schema: {
        type: "object",
        properties: { query: [Object] },
        required: [ "query" ],
        additionalProperties: false,
        "$schema": "http://json-schema.org/draft-07/schema#"
      }
    }
  ]
}

## Define the graph

We can now put it all together. We will run it first without a checkpointer:

In [66]:
import { StateGraph, END, START } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const routeMessage = (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1];
  // If no tools are called, we can finish (respond to the user)
  if (!lastMessage.tool_calls.length){
    return END;
  }
  // Otherwise if there is, we continue and call the tools
  return "action";
};

const callModel = async (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const response = await boundModel.invoke(messages);
  return { messages: [response]}
};

const workflow = new StateGraph({
  channels: State,
});

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
workflow.addNode("action", toolNode);

// Set the entrypoint as `agent`
// workflow.addEdge(START, "agent");
workflow.setEntryPoint("agent");
workflow.addConditionalEdges("agent", routeMessage);
workflow.addEdge("action", "agent");

const graph = workflow.compile()

In [79]:
let inputs = {"messages": [["user", "Hi I'm Yu, niced to meet you."]]};
for await (const output of await graph.stream(inputs, {streamMode: "updates"})) {
    console.log(output)
  console.log("-----\n");
}

{
  agent: {
    messages: [
      AIMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "It's nice to meet you too, Yu! I'm an AI assistant. I'm here to help you with any tasks or questions"... 61 more characters,
          tool_calls: [],
          invalid_tool_calls: [],
          additional_kwargs: [Object],
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "It's nice to meet you too, Yu! I'm an AI assistant. I'm here to help you with any tasks or questions"... 61 more characters,
        name: undefined,
        additional_kwargs: {
          id: "msg_01D5zErMKEqv7MHpWaCrmKPT",
          type: "message",
          role: "assistant",
          model: "claude-3-haiku-20240307",
          stop_sequence: null,
          usage: [Object],
          stop_reason: "end_turn"
        },
        response_metadata: {
          id: "msg_01D5zErMKEqv7MHpWaCrmKPT",
          model: "claude-3-haiku-2024

In [80]:
inputs = {"messages": [["user", "Remember my name?"]]};
for await (const output of await graph.stream(inputs, {streamMode: "updates"})) {
    console.log(output)
  console.log("-----\n");
}

{
  agent: {
    messages: [
      AIMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "I'm afraid I don't actually have a memory of your name. As an AI assistant created by Anthropic, I d"... 234 more characters,
          tool_calls: [],
          invalid_tool_calls: [],
          additional_kwargs: [Object],
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "I'm afraid I don't actually have a memory of your name. As an AI assistant created by Anthropic, I d"... 234 more characters,
        name: undefined,
        additional_kwargs: {
          id: "msg_01N3xMGw5E7hvvs6h2xagKCL",
          type: "message",
          role: "assistant",
          model: "claude-3-haiku-20240307",
          stop_sequence: null,
          usage: [Object],
          stop_reason: "end_turn"
        },
        response_metadata: {
          id: "msg_01N3xMGw5E7hvvs6h2xagKCL",
          model: "claude-3-haiku-20

## Add Memory

Let's try it again with a checkpointer. We will use the SqliteSaver, which will "save" checkpoints in-memory.

In [84]:
import { SqliteSaver } from "@langchain/langgraph/checkpoint";

const memory = SqliteSaver.fromConnString(":memory:") // Here we only save in-memory

TypeError: Could not resolve 'npm:@langchain/langgraph/checkpoint'.
  Caused by:
    Failed resolving package subpath './checkpoint' for '/Users/wfh/Library/Caches/deno/npm/registry.npmjs.org/@langchain/langgraph/0.0.12/package.json'
    [ERR_PACKAGE_PATH_NOT_EXPORTED] Package subpath './checkpoint' is not defined by "exports" in '/Users/wfh/Library/Caches/deno/npm/registry.npmjs.org/@langchain/langgraph/0.0.12/package.json' imported from '/Users/wfh/code/lc/langgraphjs/examples/how-tos/$deno$repl.ts'

In [85]:
const persistentGraph = workflow.compile({checkpointer: memory}); 

ReferenceError: memory is not defined

In [ ]:
let config = { configurable: { thread_id: "conversation-1" }}
inputs = {"messages": [["user", "Hi I'm Yu, niced to meet you."]]};
for await (const output of await persistentGraph.stream(inputs, {..config, streamMode: "updates"})) {
    console.log(output)
  console.log("-----\n");
}

In [ ]:
inputs = {"messages": [["user", "Remember my name?"]]};
for await (const output of await persistentGraph.stream(inputs, {..config, streamMode: "updates"})) {
    console.log(output)
  console.log("-----\n");
}

If we want to start a new conversation, we can pass in a different thread id. Poof! All the memories are gone!

In [ ]:
config = { configurable: { thread_id: "conversation-2" }}

In [ ]:
inputs = {"messages": [["user", "you forgot?"]]};
for await (const output of await persistentGraph.stream(inputs, {..config, streamMode: "updates"})) {
    console.log(output)
  console.log("-----\n");
}